In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor

# classification
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# regression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor

# Загружаем данные
## train

In [2]:
df_train = pd.read_csv('Nti2018_task2Train.csv')

cols_to_drop = ['index', 'brs_id']#, 'username', 'id']
df_train = df_train.drop(cols_to_drop, axis=1)

print(df_train.shape)
df_train.head()

(4720, 69)


,Направление,Основа,Формирующее подразделение (институт),Формирующее подразделение (департамент),Курс,Год начала,Срок освоения,Средняя оценка,Средняя оценка по первой сдаче,Код,...,ИЗ Avg,ИТ Avg,КЗ Avg,ПЗ Avg,Практ Avg,РТ Avg,Т Avg,ТЗ Avg,Тест Avg,УЗ Avg
0,uncass18ggl5g0000j12bmf3eofkej3s,сверхплановое место,"Физической культуры, спорта и молодежной политики",0,5,2013,5.0,0.00,0.00,43.00.00,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,uncass18ggl5g0000kail4d1g44ti8bs,сверхплановое место,Строительный,0,3,2015,4.0,60.49,57.65,08.00.00,...,0.986207,0.0,0.0,0.666667,0.956915,0.0,0.0,0.0,0.0,0.0
2,uncass18ggl5g0000kaou92csl390qkc,госбюджетное место,Радиоэлектроники и информационных технологий -...,Школа профессионального и академического образ...,2,2016,5.5,56.10,56.10,11.00.00,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,uncass18ggl5g0000kail4d1g44ti8bs,сверхплановое место,Строительный,0,2,2016,3.6,0.00,0.00,08.00.00,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,uncass18ggl5g0000kail4d1g44ti8bs,госбюджетное место,Строительный,0,3,2015,4.0,70.34,69.71,08.00.00,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


## test

In [3]:
# df_test = pd.read_csv('xTestFinal (1).csv')

# cols_to_drop = ['index']#, 'id', 'username']
# df_test = df_test.drop(cols_to_drop, axis=1)

# print(df_test.shape)
# df_test.head()

(1180, 43)


,Направление,Основа,Формирующее подразделение (институт),Формирующее подразделение (департамент),Курс,Год начала,Срок освоения,Средняя оценка,Средняя оценка по первой сдаче,Код,...,Биология,Физика,Академический рисунок,Обществознание,Английский язык,Вступительный экзамен по программе,Иностранный язык,История искусств,id,username
0,200,0,2,29,1,2017,2.0,0.00,0.00,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,298459,570
1,66,0,4,11,3,2015,5.0,0.00,0.00,14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160700,934
2,128,1,1,4,1,2017,5.0,0.00,0.00,27,...,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,200655,872
3,72,0,4,11,2,2016,4.0,61.26,60.57,21,...,0.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0,280247,931
4,201,0,13,0,1,2017,2.0,0.00,0.00,16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,146479,360


## Смотрим на значения признаков
## Сравниваем трейн и тест

In [20]:
print(df_train['Направление'].value_counts(normalize=True))
print(df_test['Направление'].value_counts(normalize=True))

uncass18ggl5g0000kail4d1g44ti8bs    0.152331
uncass18ggl5g0000kaio552smv9cm3o    0.043644
uncass18ggl5g0000kain7dt2j01t94k    0.041949
uncass18ggl5g0000kaq58q3ikk2rn18    0.041314
uncass18ggl5g0000kaine6felfa21hg    0.029025
uncass18ggl5g0000kaio4dnpb6qks30    0.021186
uncass18ggl5g0000klpsikee9c0cs5s    0.014407
uncass18ggl5g0000kain9utt9lu75bg    0.014407
uncass18ggl5g0000kainfe9qstfrflk    0.012288
uncass18ggl5g0000knnf22bnplgvof8    0.012076
uncass18ggl5g0000kain5r6tcncdli8    0.011229
uncass18ggl5g0000kaqvkt421bc0dtg    0.010381
uncass18ggl5g0000kaqvb84kk1t1kq0    0.010169
uncass18ggl5g0000kail4ukqj1d8dq0    0.009958
uncass18ggl5g0000kaineo7kjgemarg    0.009534
uncass18ggl5g0000kaelr9ms9n3lec4    0.009322
uncass18ggl5g0000kastgtvio9g2a1c    0.009322
uncass18ggl5g0000kaingjml6g5acs8    0.008686
uncass18ggl5g0000kaio6g0ms7deiag    0.008686
uncass18ggl5g0000kaqsplbivu3ie48    0.008263
uncass18ggl5g0000kaqttcnm6a7dl7k    0.007839
uncass18ggl5g0000kainak4oc18p6l0    0.007839
uncass18gg

In [21]:
print(df_train['Направление.1'].value_counts(normalize=True))
print(df_test['Направление.1'].value_counts(normalize=True))

Строительство                                                                0.166737
Металлургия                                                                  0.069703
Электроэнергетика и электротехника                                           0.050636
Технологические машины и оборудование                                        0.049364
Конструкторско-технологическое обеспечение машиностроительных производств    0.030932
Энергетическое машиностроение                                                0.026483
Менеджмент                                                                   0.025847
Химическая технология                                                        0.025847
Автоматизация технологических процессов и производств                        0.023305
Экономика                                                                    0.021822
Информационные системы и технологии                                          0.021822
Теплоэнергетика и теплотехника                        

In [18]:
print(df_train['Уровень'].value_counts(normalize=True))
print(df_test['Уровень'].value_counts(normalize=True))

Бакалавр                  0.726907
Магистр                   0.097458
Прикладной бакалавриат    0.083898
Специалист                0.073941
Аспирант                  0.017797
Name: Уровень, dtype: float64
1    0.733051
2    0.094068
3    0.089831
4    0.070339
0    0.012712
Name: Уровень, dtype: float64


In [19]:
print(df_train['Уровеь'].value_counts(normalize=True))
print(df_test['Уровеь'].value_counts(normalize=True))

62    0.726907
68    0.097458
61    0.083898
65    0.073941
70    0.017797
Name: Уровеь, dtype: float64
62    0.733051
68    0.094068
61    0.089831
65    0.070339
70    0.012712
Name: Уровеь, dtype: float64


# Разделяем на X и y

In [4]:
y_cols = df_train.columns[df_train.columns.str.contains('Avg')]#.tolist()
X_train = df_train.drop(y_cols, axis=1)
y_train = df_train[y_cols]
print(X_train.shape, y_train.shape)

(4720, 43) (4720, 26)


# Подготавливаем признаки

In [7]:
categorical_cols = [
    'Основа',
    'Формирующее подразделение (институт)',
    'Формирующее подразделение (департамент)',
    'Код',
    'ОКСО',
    'Уровеь',
    'Форма',
    'Олимпиада',
    'Год начала',
    'Курс',
]
X_train_cat = X_train[categorical_cols]
print(X_train_cat.shape)

X_train_cat = pd.get_dummies(X_train_cat, columns=categorical_cols, drop_first=False)
print(X_train_cat.shape)
X_train_cat.head()

(4720, 10)
(4720, 308)


,Основа_госбюджетное место,Основа_сверхплановое место,Формирующее подразделение (институт)_Высшая школа экономики и менеджмента,Формирующее подразделение (институт)_Государственного управления и предпринимательства,Формирующее подразделение (институт)_Естественных наук и математики,Формирующее подразделение (институт)_Нижнетагильский технологический,Формирующее подразделение (институт)_Новых материалов и технологий,Формирующее подразделение (институт)_Радиоэлектроники и информационных технологий - РТФ,Формирующее подразделение (институт)_Строительный,Формирующее подразделение (институт)_Уральский гуманитарный институт,...,Год начала_2014,Год начала_2015,Год начала_2016,Год начала_2017,Курс_1,Курс_2,Курс_3,Курс_4,Курс_5,Курс_6
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0
2,1,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
3,0,1,0,0,0,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,0
4,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0


In [9]:
ege_cols = [
    'Физическая культура',
    'Рисунок',
    'Химия',
    'География',
    'Информатика и ИКТ',
    'История',
    'Математика',
    'Композиционный рисунок',
    'Композиция',
    'Рисунок и композиция',
    'Русский язык',
    'Творческий конкурс',
    'Собеседование',
    'Филология (отечественная)',
    'Творческое сочинение',
    'Литература',
    'Филология',
    'Биология',
    'Физика',
    'Академический рисунок',
    'Обществознание',
    'Английский язык',
    'Вступительный экзамен по программе',
    'Иностранный язык',
    'История искусств',
]

numerical_cols = [
    'Курс',
    'Срок освоения',
    'Средняя оценка ',
    'Средняя оценка по первой сдаче',
]

X_train_cont = X_train[ege_cols + numerical_cols]
print(X_train_cont.shape)
X_train_cont.head()

(4720, 29)


,Физическая культура,Рисунок,Химия,География,Информатика и ИКТ,История,Математика,Композиционный рисунок,Композиция,Рисунок и композиция,...,Академический рисунок,Обществознание,Английский язык,Вступительный экзамен по программе,Иностранный язык,История искусств,Курс,Срок освоения,Средняя оценка,Средняя оценка по первой сдаче
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5,5.0,0.00,0.00
1,0.0,0.0,0.0,0.0,0.0,0.0,55.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,4.0,60.49,57.65
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,5.5,56.10,56.10
3,0.0,0.0,0.0,0.0,0.0,0.0,48.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,3.6,0.00,0.00
4,0.0,0.0,0.0,0.0,0.0,0.0,70.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,4.0,70.34,69.71


In [12]:
X_train_bin = X_train[ege_cols] != 0
print(X_train_bin.shape)
X_train_bin.head()

(4720, 25)


,Физическая культура,Рисунок,Химия,География,Информатика и ИКТ,История,Математика,Композиционный рисунок,Композиция,Рисунок и композиция,...,Литература,Филология,Биология,Физика,Академический рисунок,Обществознание,Английский язык,Вступительный экзамен по программе,Иностранный язык,История искусств
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False


In [13]:
X_train = pd.concat([X_train_cat, X_train_cont, X_train_bin], axis=1)
print(X_train.shape)

(4720, 362)


# Обучаем и оцениваем модель
## Предсказываем, проходил ли курс или нет

In [29]:
def print_score(y_bin, y_bin_pred):
    for col in y_cols:
        print('{}'.format(col))
        top_class_rate = y_train_bin[col].value_counts(normalize=True)[0]
        acc = accuracy_score(y_train_bin[col], y_train_bin_pred[col])
        prec = precision_score(y_train_bin[col], y_train_bin_pred[col])
        rec = recall_score(y_train_bin[col], y_train_bin_pred[col])
        print('\t{:.3} {:.3}'.format(acc, top_class_rate))
        print('\t{:.3} {:.3}'.format(prec, rec))

In [30]:
y_train_bin = y_train != 0
y_train_bin.head()

,Final Avg,Final Exam Avg,HW Avg,TC Avg,ig Avg,metrology Avg,ng Avg,sootv Avg,stand Avg,termodin Avg,...,ИЗ Avg,ИТ Avg,КЗ Avg,ПЗ Avg,Практ Avg,РТ Avg,Т Avg,ТЗ Avg,Тест Avg,УЗ Avg
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,True,False,False,True,True,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [34]:
model = LogisticRegression(n_jobs=1)
mo_model = MultiOutputClassifier(model)
%time y_train_bin_pred = cross_val_predict(mo_model, X_train, y_train_bin, cv=3)
y_train_bin_pred = pd.DataFrame(data=y_train_bin_pred, columns=y_train.columns)

print_score(y_train_bin, y_train_bin_pred)

CPU times: user 17.7 s, sys: 412 ms, total: 18.1 s
Wall time: 10.6 s
Final Avg
	0.953 0.949
	0.604 0.239
Final Exam Avg
	0.962 0.963
	0.25 0.0171
HW Avg
	0.988 0.988
	0.0 0.0
TC Avg
	0.979 0.979
	0.0 0.0
ig Avg
	0.992 0.992
	0.0 0.0
metrology Avg
	0.958 0.946
	0.69 0.42
ng Avg
	0.986 0.986
	0.0 0.0
sootv Avg
	0.962 0.953
	0.642 0.43
stand Avg
	0.963 0.951
	0.684 0.446
termodin Avg
	0.997 0.997
	0.0 0.0
БП Avg
	0.892 0.878
	0.641 0.272
ДЗ Avg
	0.876 0.844
	0.66 0.434
ДР Avg
	0.989 0.989
	0.0 0.0
Дз1 Avg
	0.997 0.997
	0.0 0.0
Дз2 Avg
	0.976 0.976
	0.0 0.0
З Avg
	0.979 0.979
	0.0 0.0
ИЗ Avg
	0.94 0.943
	0.176 0.0111
ИТ Avg
	0.919 0.92
	0.357 0.0132
КЗ Avg
	0.97 0.971
	0.2 0.00719
ПЗ Avg
	0.946 0.947
	0.278 0.0201
Практ Avg
	0.949 0.949
	0.571 0.0165
РТ Avg
	0.999 0.999
	0.0 0.0
Т Avg
	0.866 0.832
	0.663 0.414
ТЗ Avg
	0.923 0.924
	0.222 0.00557
Тест Avg
	0.952 0.952
	0.25 0.00444
УЗ Avg
	0.908 0.911
	0.355 0.0526


/home/Maslovsky/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [39]:
# model = KNeighborsClassifier(n_jobs=-1)
# mo_model = MultiOutputClassifier(model)
# %time y_train_bin_pred = cross_val_predict(mo_model, X_train, y_train_bin, cv=3)
# y_train_bin_pred = pd.DataFrame(data=y_train_bin_pred, columns=y_train.columns)

# print_score(y_train_bin, y_train_bin_pred)

In [38]:
model = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
mo_model = MultiOutputClassifier(model)
%time y_train_bin_pred = cross_val_predict(mo_model, X_train, y_train_bin, cv=3)
y_train_bin_pred = pd.DataFrame(data=y_train_bin_pred, columns=y_train.columns)

print_score(y_train_bin, y_train_bin_pred)

CPU times: user 8min 51s, sys: 48 s, total: 9min 39s
Wall time: 3min 31s
Final Avg
	0.951 0.949
	0.531 0.35
Final Exam Avg
	0.963 0.963
	0.493 0.206
HW Avg
	0.987 0.988
	0.0 0.0
TC Avg
	0.975 0.979
	0.207 0.06
ig Avg
	0.99 0.992
	0.0 0.0
metrology Avg
	0.958 0.946
	0.658 0.475
ng Avg
	0.982 0.986
	0.167 0.0746
sootv Avg
	0.965 0.953
	0.657 0.52
stand Avg
	0.964 0.951
	0.667 0.524
termodin Avg
	0.997 0.997
	0.0 0.0
БП Avg
	0.889 0.878
	0.556 0.438
ДЗ Avg
	0.869 0.844
	0.599 0.497
ДР Avg
	0.987 0.989
	0.0769 0.0192
Дз1 Avg
	0.995 0.997
	0.0 0.0
Дз2 Avg
	0.967 0.976
	0.0 0.0
З Avg
	0.975 0.979
	0.212 0.0693
ИЗ Avg
	0.92 0.943
	0.147 0.0812
ИТ Avg
	0.893 0.92
	0.174 0.0899
КЗ Avg
	0.966 0.971
	0.256 0.0791
ПЗ Avg
	0.924 0.947
	0.196 0.141
Практ Avg
	0.926 0.949
	0.0496 0.0248
РТ Avg
	0.999 0.999
	0.0 0.0
Т Avg
	0.856 0.832
	0.588 0.476
ТЗ Avg
	0.904 0.924
	0.134 0.0474
Тест Avg
	0.938 0.952
	0.147 0.0622
УЗ Avg
	0.894 0.911
	0.304 0.151


## Предсказываем среднюю оценку за курс

In [42]:
def get_score(y, y_pred):
    mse = mean_squared_error(y, y_pred)
    msez = mean_squared_error(y, np.zeros(y.shape))
    return 10 * (1 - mse / msez)

print(get_score(y_train, np.zeros(y_train.shape)))
print(get_score(y_train, y_train))

0.0
10.0


In [47]:
model = LinearRegression()
mo_model = MultiOutputRegressor(model)
%time y_train_pred = cross_val_predict(mo_model, X_train, y_train, cv=3)
y_train_pred = pd.DataFrame(data=y_train_pred, columns=y_train.columns)
y_train_pred[~y_train_bin_pred] = 0

get_score(y_train, y_train_pred)

CPU times: user 1min 27s, sys: 3.02 s, total: 1min 30s
Wall time: 5.62 s


1.6457898618788247

In [48]:
model = RandomForestRegressor(n_estimators=1000, n_jobs=-1)
mo_model = MultiOutputRegressor(model)
%time y_train_pred = cross_val_predict(mo_model, X_train, y_train, cv=3)
y_train_pred = pd.DataFrame(data=y_train_pred, columns=y_train.columns)
y_train_pred[~y_train_bin_pred] = 0

get_score(y_train, y_train_pred)

CPU times: user 1h 30min 27s, sys: 15.6 s, total: 1h 30min 42s
Wall time: 5min 38s


0.89864619932307099